In [8]:
from model import UNET
from dataset import UNET_Dataset
import albumentations as A

from albumentations.pytorch import ToTensorV2
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
from torch.nn.functional import sigmoid

IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128

In [9]:
model = UNET.load_from_checkpoint("lightning_logs\\version_9\\checkpoints\\epoch=124-step=9750.ckpt", lr = 1e-3, weight_decay=1e-3).cuda()

In [10]:
data = UNET_Dataset("Dataset/test", A.Compose(
            [
                A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
                A.Normalize(
                    mean=[0.0, 0.0, 0.0],
                    std=[1.0, 1.0, 1.0],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ],
        ))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [15]:
for i, (img, seg) in enumerate(data):
    output = model(img.unsqueeze(0).cuda())
    preds = sigmoid(output)
    preds = (preds >= 0.5).float()

    fig = plt.figure(figsize=(6, 3))

    fig.add_subplot(1, 3, 1)
    plt.imshow(ToPILImage()(img))
    plt.axis("off")
    plt.title("Input")

    fig.add_subplot(1, 3, 2)
    plt.imshow(ToPILImage()(seg))
    plt.axis("off")
    plt.title("Target")

    fig.add_subplot(1, 3, 3)
    plt.imshow(ToPILImage()(preds.squeeze(0)))
    plt.axis("off")
    plt.title("Output")
    
    plt.savefig("Outputs/" + str(i) + ".jpg")
    plt.close()

KeyError: 34

In [ ]:
from data_module import DataModule
import pytorch_lightning as pl

BATCH_SIZE = 32
NUM_EPOCHS = 256
NUM_WORKERS = 2
PIN_MEMORY = True
DIR = "D:\\Machine Learning\\U-Net Segmentation\\Dataset"

data_module = DataModule(DIR, NUM_WORKERS, BATCH_SIZE, PIN_MEMORY)
data_module.setup("validate")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=NUM_EPOCHS,
    devices=[0],
    log_every_n_steps=1,
)
trainer.test(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8956891298294067
       dice_score            0.79505455493927
        test_loss           0.7620352506637573
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.7620352506637573,
  'dice_score': 0.79505455493927,
  'accuracy': 0.8956891298294067}]

In [ ]:
import torch
torch.save(
    model.state_dict(),
    "D:\\Machine Learning\\U-Net Segmentation\\saved_models\\model.pth"
)